In [83]:
from minepy import MINE
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectFromModel, SelectKBest, VarianceThreshold
import warnings

In [2]:
pd.set_option('display.max_columns', None)
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('G:\IDS2018\datasets\Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv')

# 数据预处理

In [4]:
numerical_fea = list(data.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea, list(data.columns)))

In [5]:
def get_numerical_serial_fea(data, feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique()
        if temp <= 10:
            numerical_noserial_fea.append(fea)
        else:
            numerical_serial_fea.append(fea)
    return numerical_serial_fea, numerical_noserial_fea
numerical_serial_fea, numerical_noserial_fea = get_numerical_serial_fea(data, numerical_fea)

In [36]:
data.isnull().sum().to_dict()

{'Dst Port': 0,
 'Protocol': 0,
 'Timestamp': 0,
 'Flow Duration': 0,
 'Tot Fwd Pkts': 0,
 'Tot Bwd Pkts': 0,
 'TotLen Fwd Pkts': 0,
 'TotLen Bwd Pkts': 0,
 'Fwd Pkt Len Max': 0,
 'Fwd Pkt Len Min': 0,
 'Fwd Pkt Len Mean': 0,
 'Fwd Pkt Len Std': 0,
 'Bwd Pkt Len Max': 0,
 'Bwd Pkt Len Min': 0,
 'Bwd Pkt Len Mean': 0,
 'Bwd Pkt Len Std': 0,
 'Flow Byts/s': 8027,
 'Flow Pkts/s': 8027,
 'Flow IAT Mean': 0,
 'Flow IAT Std': 0,
 'Flow IAT Max': 0,
 'Flow IAT Min': 0,
 'Fwd IAT Tot': 0,
 'Fwd IAT Mean': 0,
 'Fwd IAT Std': 0,
 'Fwd IAT Max': 0,
 'Fwd IAT Min': 0,
 'Bwd IAT Tot': 0,
 'Bwd IAT Mean': 0,
 'Bwd IAT Std': 0,
 'Bwd IAT Max': 0,
 'Bwd IAT Min': 0,
 'Fwd PSH Flags': 0,
 'Bwd PSH Flags': 0,
 'Fwd URG Flags': 0,
 'Bwd URG Flags': 0,
 'Fwd Header Len': 0,
 'Bwd Header Len': 0,
 'Fwd Pkts/s': 0,
 'Bwd Pkts/s': 0,
 'Pkt Len Min': 0,
 'Pkt Len Max': 0,
 'Pkt Len Mean': 0,
 'Pkt Len Std': 0,
 'Pkt Len Var': 0,
 'FIN Flag Cnt': 0,
 'SYN Flag Cnt': 0,
 'RST Flag Cnt': 0,
 'PSH Flag Cnt': 0,
 

In [6]:
data[numerical_serial_fea] = data[numerical_serial_fea].fillna(data[numerical_serial_fea].mean())

In [9]:
data.isnull().sum().to_dict()

{'Dst Port': 0,
 'Protocol': 0,
 'Timestamp': 0,
 'Flow Duration': 0,
 'Tot Fwd Pkts': 0,
 'Tot Bwd Pkts': 0,
 'TotLen Fwd Pkts': 0,
 'TotLen Bwd Pkts': 0,
 'Fwd Pkt Len Max': 0,
 'Fwd Pkt Len Min': 0,
 'Fwd Pkt Len Mean': 0,
 'Fwd Pkt Len Std': 0,
 'Bwd Pkt Len Max': 0,
 'Bwd Pkt Len Min': 0,
 'Bwd Pkt Len Mean': 0,
 'Bwd Pkt Len Std': 0,
 'Flow Byts/s': 0,
 'Flow Pkts/s': 0,
 'Flow IAT Mean': 0,
 'Flow IAT Std': 0,
 'Flow IAT Max': 0,
 'Flow IAT Min': 0,
 'Fwd IAT Tot': 0,
 'Fwd IAT Mean': 0,
 'Fwd IAT Std': 0,
 'Fwd IAT Max': 0,
 'Fwd IAT Min': 0,
 'Bwd IAT Tot': 0,
 'Bwd IAT Mean': 0,
 'Bwd IAT Std': 0,
 'Bwd IAT Max': 0,
 'Bwd IAT Min': 0,
 'Fwd PSH Flags': 0,
 'Bwd PSH Flags': 0,
 'Fwd URG Flags': 0,
 'Bwd URG Flags': 0,
 'Fwd Header Len': 0,
 'Bwd Header Len': 0,
 'Fwd Pkts/s': 0,
 'Bwd Pkts/s': 0,
 'Pkt Len Min': 0,
 'Pkt Len Max': 0,
 'Pkt Len Mean': 0,
 'Pkt Len Std': 0,
 'Pkt Len Var': 0,
 'FIN Flag Cnt': 0,
 'SYN Flag Cnt': 0,
 'RST Flag Cnt': 0,
 'PSH Flag Cnt': 0,
 'ACK F

In [7]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'],format='%d/%m/%Y %H:%M:%S')

In [8]:
data['Label'].replace(to_replace='Benign', value=0, inplace=True)
data['Label'].replace(to_replace='DoS attacks-GoldenEye', value=1, inplace=True)
data['Label'].replace(to_replace='DoS attacks-Slowloris', value=1, inplace=True)

# 异常值处理

In [10]:
def find_outliers_by_3segama(data, fea):
    data_std = np.std(data[fea])
    data_mean = np.mean(data[fea])
    outliers_cut_off = 3 * data_std
    lower_rule = data_mean - outliers_cut_off
    upper_rule = data_mean + outliers_cut_off
    data[fea+'_outliers'] = data[fea].apply(lambda x: str('异常值') if x > upper_rule or x < lower_rule else '正常值')
    return data

In [27]:
for fea in numerical_serial_fea:
    data = find_outliers_by_3segama(data, fea)
    print(data[fea+'_outliers'].value_counts())
    print(data.groupby(fea+'_outliers')['Label'].sum())
    print('*'*10)

正常值    1039789
异常值       8786
Name: Dst Port_outliers, dtype: int64
Dst Port_outliers
异常值        0
正常值    52498
Name: Label, dtype: int64
**********
正常值    991764
异常值     56811
Name: Flow Duration_outliers, dtype: int64
Flow Duration_outliers
异常值      280
正常值    52218
Name: Label, dtype: int64
**********
正常值    1046745
异常值       1830
Name: Tot Fwd Pkts_outliers, dtype: int64
Tot Fwd Pkts_outliers
异常值        0
正常值    52498
Name: Label, dtype: int64
**********
正常值    1046952
异常值       1623
Name: Tot Bwd Pkts_outliers, dtype: int64
Tot Bwd Pkts_outliers
异常值        0
正常值    52498
Name: Label, dtype: int64
**********
正常值    1048549
异常值         26
Name: TotLen Fwd Pkts_outliers, dtype: int64
TotLen Fwd Pkts_outliers
异常值        0
正常值    52498
Name: Label, dtype: int64
**********
正常值    1047070
异常值       1505
Name: TotLen Bwd Pkts_outliers, dtype: int64
TotLen Bwd Pkts_outliers
异常值        0
正常值    52498
Name: Label, dtype: int64
**********
正常值    1038021
异常值      10554
Name: Fwd Pkt Len Max_ou

正常值    1034351
异常值      14224
Name: Active Std_outliers, dtype: int64
Active Std_outliers
异常值     4935
正常值    47563
Name: Label, dtype: int64
**********
正常值    1032734
异常值      15841
Name: Active Max_outliers, dtype: int64
Active Max_outliers
异常值     5557
正常值    46941
Name: Label, dtype: int64
**********
正常值    1035242
异常值      13333
Name: Active Min_outliers, dtype: int64
Active Min_outliers
异常值     5463
正常值    47035
Name: Label, dtype: int64
**********
正常值    996157
异常值     52418
Name: Idle Mean_outliers, dtype: int64
Idle Mean_outliers
异常值     4229
正常值    48269
Name: Label, dtype: int64
**********
正常值    1037445
异常值      11130
Name: Idle Std_outliers, dtype: int64
Idle Std_outliers
异常值     4788
正常值    47710
Name: Label, dtype: int64
**********
正常值    988121
异常值     60454
Name: Idle Max_outliers, dtype: int64
Idle Max_outliers
异常值     8686
正常值    43812
Name: Label, dtype: int64
**********
正常值    996490
异常值     52085
Name: Idle Min_outliers, dtype: int64
Idle Min_outliers
异常值     4277

# 特征选择

In [87]:
pd.DataFrame(VarianceThreshold(threshold=3).fit_transform(data[numerical_serial_fea])) # Down/Up Ratio is deleted

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57
0,0.0,112641158.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.026633,5.632058e+07,7.042784e+02,56321077.0,56320081.0,112641158.0,5.632058e+07,7.042784e+02,56321077.0,56320081.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.026633,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.000000,0.0,0.0,56320579.0,7.042784e+02,56321077.0,56320081.0
1,22.0,37366762.0,14.0,12.0,2168.0,2993.0,712.0,0.0,154.857143,254.855270,976.0,0.0,249.416667,395.929392,1.381174e+02,0.695806,1.494670e+06,3.894924e+06,15617415.0,7.0,37366762.0,2.874366e+06,5.104444e+06,15617415.0,40.0,37366730.0,3.396975e+06,5.564224e+06,15936762.0,893.0,456.0,416.0,0.374665,0.321141,0.0,976.0,191.148148,320.122898,102478.669516,198.500000,154.857143,249.416667,14.0,2168.0,12.0,2993.0,29200.0,232.0,8.0,32.0,1024353.0,649038.754495,1601183.0,321569.0,11431221.0,3.644991e+06,15617415.0,8960247.0
2,47514.0,543.0,2.0,0.0,64.0,0.0,64.0,0.0,32.000000,45.254834,0.0,0.0,0.000000,0.000000,1.178637e+05,3683.241252,5.430000e+02,0.000000e+00,543.0,543.0,543.0,5.430000e+02,0.000000e+00,543.0,543.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,64.0,0.0,3683.241252,0.000000,0.0,64.0,42.666667,36.950417,1365.333333,64.000000,32.000000,0.000000,2.0,64.0,0.0,0.0,244.0,-1.0,0.0,32.0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0
3,0.0,112640703.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.026633,5.632035e+07,3.669884e+02,56320611.0,56320092.0,112640703.0,5.632035e+07,3.669884e+02,56320611.0,56320092.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.026633,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.000000,0.0,0.0,56320351.5,3.669884e+02,56320611.0,56320092.0
4,0.0,112640874.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.026633,5.632044e+07,7.198347e+02,56320946.0,56319928.0,112640874.0,5.632044e+07,7.198347e+02,56320946.0,56319928.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.026633,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.000000,0.0,0.0,56320437.0,7.198347e+02,56320946.0,56319928.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,50111.0,22.0,3.0,0.0,31.0,0.0,31.0,0.0,10.333333,17.897858,0.0,0.0,0.000000,0.000000,1.409091e+06,136363.636364,1.100000e+01,1.555635e+01,22.0,0.0,22.0,1.100000e+01,1.555635e+01,22.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,60.0,0.0,136363.636364,0.000000,0.0,31.0,7.750000,15.500000,240.250000,10.333333,10.333333,0.000000,3.0,31.0,0.0,0.0,64.0,-1.0,1.0,20.0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0
1048571,443.0,54682783.0,5.0,1.0,123.0,46.0,46.0,0.0,24.600000,23.276598,46.0,46.0,46.000000,0.000000,3.090552e+00,0.109724,1.093656e+07,2.436610e+07,54523813.0,17.0,54682783.0,1.367070e+07,2.723551e+07,54523813.0,17.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,100.0,20.0,0.091436,0.018287,0.0,46.0,30.714286,21.684974,470.238095,35.833333,24.600000,46.000000,5.0,123.0,1.0,46.0,256.0,180.0,2.0,20.0,158783.0,0.000000,158783.0,158783.0,54523813.0,0.000000e+00,54523813.0,54523813.0
1048572,443.0,53682093.0,5.0,1.0,123.0,46.0,46.0,0.0,24.600000,23.276598,46.0,46.0,46.000000,0.000000,3.148163e+00,0.111769,1.073642e+07,2.386193e+07,53421756.0,16.0,53682093.0,1.342052e+07,2.666777e+07,53421756.0,16.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,100.0,20.0,0.093141,0.018628,0.0,46.0,30.714286,21.684974,470.238095,35.833333,24.600000,46.000000,5.0,123.0,1.0,46.0,254.0,180.0,2.0,20.0,259719.0,0.000000,259719.0,259719.0,53421756.

In [117]:
pearsonr_result = []
for fea in numerical_serial_fea:
    pearsonr_result.append((fea, pearsonr(data[fea], data['Label'])))
sorted(pearsonr_result, key=lambda x: x[1][0], reverse=True)[:10]

[('Fwd Seg Size Min', (0.4827216248545034, 0.0)),
 ('Bwd IAT Mean', (0.30192251129739955, 0.0)),
 ('Init Fwd Win Byts', (0.2684559690822708, 0.0)),
 ('Bwd IAT Min', (0.2500177147030524, 0.0)),
 ('Flow IAT Std', (0.21858340497356774, 0.0)),
 ('Bwd IAT Max', (0.19606408350148533, 0.0)),
 ('Bwd IAT Std', (0.18719441138249784, 0.0)),
 ('Idle Max', (0.18100280839192964, 0.0)),
 ('Fwd Pkt Len Std', (0.16886516326958462, 0.0)),
 ('Idle Std', (0.15481787043716266, 0.0))]